In [6]:
import pandas as pd
import os.path

In [7]:
ROOT_DIR = os.path.abspath(os.pardir)
BASE_DIR = os.path.join(ROOT_DIR, 'data')

In [8]:
data = os.path.join(BASE_DIR, 'processed', 'casas.csv')
df = pd.read_csv(data)

In [9]:
df.head()

,tamanho,ano,garagem,preco
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [10]:
os.pardir

'..'

In [11]:
X = df.drop('preco', axis=1)
y = df['preco'].copy()

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
X_train.shape

(1022, 3)

In [14]:
X_test.shape

(438, 3)

In [2]:
import mlflow

In [3]:
# iniciar um projeto
mlflow.set_experiment('house-prices-eda')

2022/04/27 20:36:38 INFO mlflow.tracking.fluent: Experiment with name 'house-prices-eda' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/weslley/Desktop/MLEngineer-Studies/MlFlow-Alura/mlflow/notebooks/mlruns/1', experiment_id='1', lifecycle_stage='active', name='house-prices-eda', tags={}>

In [4]:
mlflow.start_run()

<ActiveRun: >

## Linear Regression

In [15]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [16]:
mlflow.sklearn.log_model(lr, 'lr')

/home/weslley/Desktop/MLEngineer-Studies/MlFlow-Alura/.venv_mlflow/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


ModelInfo(artifact_path='lr', flavors={'python_function': {'model_path': 'model.pkl', 'loader_module': 'mlflow.sklearn', 'python_version': '3.9.7', 'env': 'conda.yaml'}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.0.2', 'serialization_format': 'cloudpickle', 'code': None}}, model_uri='runs:/57082c91aca34e219d56cbfd78522781/lr', model_uuid='94641f4355364943898dac2fef6c104b', run_id='57082c91aca34e219d56cbfd78522781', saved_input_example_info=None, signature_dict=None, utc_time_created='2022-04-27 23:40:29.414436', mlflow_version='1.25.1')

In [20]:
lr_predicted = lr.predict(X_test)

In [21]:
X_test.iloc[0]

tamanho      99.0
ano        1963.0
garagem       1.0
Name: 892, dtype: float64

In [22]:
y_test.iloc[0]

154500

In [23]:
from sklearn.metrics import mean_squared_error, r2_score
import math

In [24]:
mse = mean_squared_error(y_test, lr_predicted)
rmse = math.sqrt(mse)
r2 = r2_score(y_test, lr_predicted)

mlflow.log_metric('mse', mse)
mlflow.log_metric('rmse', rmse)
mlflow.log_metric('r2', r2)
print(mse, rmse, r2)

2078666917.9289913 45592.399782518485 0.7021153642898048


In [26]:
mlflow.end_run()

In [25]:
from xgboost import XGBRFClassifier, XGBRegressor

In [28]:
xgb_params = {
    'learning_rate':0.2,
    'n_estimators':50,
    'random_state':42
}

with mlflow.start_run():
    xgb = XGBRegressor(**xgb_params)
    xgb.fit(X_train, y_train)
    mlflow.xgboost.log_model(xgb, 'xgboost')
    
    xgb_predicted = xgb.predict(X_test)
    
    mse = mean_squared_error(y_test, xgb_predicted)
    rmse = math.sqrt(mse)
    r2 = r2_score(y_test, xgb_predicted)

    mlflow.log_metric('mse', mse)
    mlflow.log_metric('rmse', rmse)
    mlflow.log_metric('r2', r2)
    print(mse, rmse, r2)

1386727460.1346002 37238.789724353286 0.8012741720529797


1550816053.129738 39380.40189141977 0.7777593557411607


## Consumindo API do experimento

In [29]:
mlflow.get_experiment_by_name('house-prices-eda').experiment_id

<Experiment: artifact_location='file:///home/weslley/Desktop/MLEngineer-Studies/MlFlow-Alura/mlflow/notebooks/mlruns/1', experiment_id='1', lifecycle_stage='active', name='house-prices-eda', tags={}>

In [34]:
id_experiment = mlflow.get_experiment_by_name('house-prices-eda').experiment_id
id_experiment

'1'

In [35]:
mlflow.list_run_infos(id_experiment)

[<RunInfo: artifact_uri='file:///home/weslley/Desktop/MLEngineer-Studies/MlFlow-Alura/mlflow/notebooks/mlruns/1/9d269fd4b8c349edafb9eae9693dc196/artifacts', end_time=1651104179846, experiment_id='1', lifecycle_stage='active', run_id='9d269fd4b8c349edafb9eae9693dc196', run_uuid='9d269fd4b8c349edafb9eae9693dc196', start_time=1651104176894, status='FINISHED', user_id='weslley'>,
 <RunInfo: artifact_uri='file:///home/weslley/Desktop/MLEngineer-Studies/MlFlow-Alura/mlflow/notebooks/mlruns/1/deec634740e84eba9dd2551eb8f91ebb/artifacts', end_time=1651104045795, experiment_id='1', lifecycle_stage='active', run_id='deec634740e84eba9dd2551eb8f91ebb', run_uuid='deec634740e84eba9dd2551eb8f91ebb', start_time=1651104042394, status='FINISHED', user_id='weslley'>,
 <RunInfo: artifact_uri='file:///home/weslley/Desktop/MLEngineer-Studies/MlFlow-Alura/mlflow/notebooks/mlruns/1/57082c91aca34e219d56cbfd78522781/artifacts', end_time=1651103422816, experiment_id='1', lifecycle_stage='active', run_id='57082c91

In [36]:
run_id = mlflow.list_run_infos(id_experiment)[-1].run_id
run_id

'57082c91aca34e219d56cbfd78522781'

In [37]:
mlflow.get_run(run_id)

<Run: data=<RunData: metrics={'mse': 2078666917.9289913,
 'r2': 0.7021153642898048,
 'rmse': 45592.399782518485}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "57082c91aca34e219d56cbfd78522781", '
                             '"artifact_path": "lr", "utc_time_created": '
                             '"2022-04-27 23:40:29.414436", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.9.7", "env": "conda.yaml"}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"94641f4355364943898dac2fef6c104b", '
                             '"mlflow_version": "1.25.1"}]',
 'mlflow.source.gi